<img src="./images/DLI_Header.png" style="width: 400px;">

# Képosztályozás az MNIST adathalmazzal

Ebben a szakaszban a mélytanulás "Hello World"-jét fogjuk elvégezni: egy mélytanulási modellt képzünk ki a kézzel írt számjegyek helyes osztályozására.

* Értsd meg, hogy a mélytanulás hogyan képes megoldani olyan problémákat, amelyeket a hagyományos programozási módszerek nem tudnak megoldani.
* Ismerje meg az [MNSIT kézzel írt számjegyek adathalmazát](http://yann.lecun.com/exdb/mnist/)
* A [Keras API](https://keras.io/) használata az MNIST adathalmaz betöltéséhez és a képzéshez való előkészítéséhez
* Hozzon létre egy egyszerű neurális hálózatot a képosztályozás elvégzéséhez
* A neurális hálózat képzése az előkészített MNIST-adatkészlet segítségével
* Figyelje meg a betanított neurális hálózat teljesítményét.

## A probléma: Képosztályozás

A hagyományos programozás során a programozó képes szabályokat és feltételeket megfogalmazni a kódjában, amelyeket a programja a megfelelő módon történő cselekvéshez használhat. Ez a megközelítés továbbra is kivételesen jól működik a legkülönbözőbb problémák esetében.

A képosztályozás, amely arra kéri a programot, hogy helyesen soroljon be egy olyan képet a megfelelő osztályba, amelyet még soha nem látott, a hagyományos programozási technikákkal szinte lehetetlen megoldani. Hogyan tudná egy programozó meghatározni azokat a szabályokat és feltételeket, amelyekkel helyesen osztályozhatná a képek óriási választékát, különösen olyan képek figyelembevételével, amelyeket még soha nem látott?

## A megoldás: Deep Learning

A mélytanulás a próbálgatással történő mintafelismerésben jeleskedik. Ha egy mély neurális hálózatot elegendő adattal betanítunk, és a hálózatnak a képzésen keresztül visszajelzést adunk a teljesítményéről, a hálózat hatalmas mennyiségű iterációval képes azonosítani a saját feltételrendszerét, amely alapján a megfelelő módon tud cselekedni.

## Az MNIST adathalmaz

A mélytanulás történetében jelentős előrelépés volt a [MNSIT adathalmaz] (http://yann.lecun.com/exdb/mnist/) pontos képosztályozása, amely egy 70 000 szürkeárnyalatos képet tartalmazó gyűjtemény kézzel írt számjegyekről 0-tól 9-ig. Míg ma a probléma triviálisnak számít, a képosztályozás elvégzése az MNIST-tel egyfajta "Hello World" lett a mélytanulás számára.

Here are 40 of the images included in the MNIST dataset:

<img src="./images/mnist1.png" style="width: 600px;">

## Tanítési és validálási adatok és címkék

Amikor képekkel dolgozunk a mélytanuláshoz, szükségünk van magukra a képekre, amelyeket általában "X"-ként jelölünk, valamint a képek helyes [címkékre] (https://developers.google.com/machine-learning/glossary#label), amelyeket általában "Y"-ként jelölünk. Továbbá, szükségünk van `X` és `Y` értékekre a modell *kiképzéséhez*, majd egy külön `X` és `Y` értékkészletre a modell teljesítményének *érvényesítéséhez* a modell kiképzése után. Ezért az MNIST-adatkészlethez 4 szegmensre van szükségünk:

1. `x_train`: A neurális hálózat képzéséhez használt képek
2. `y_train`: Az "x_train" képek helyes címkézése, a modell előrejelzéseinek értékelésére szolgál a képzés során.
3. `x_valid`: A modell teljesítményének validálására elkülönített képek a modell betanítása után.
4. `y_valid`: Az "x_valid" képek helyes címkék, amelyek a modell előrejelzéseinek értékelésére szolgálnak a modell betanítása után.

Az adatok elemzésre való előkészítésének folyamatát [Data Engineering]-nek nevezzük (https://medium.com/@rchang/a-beginners-guide-to-data-engineering-part-i-4227c5c457d7). Ha többet szeretne megtudni a képzési adatok és a validációs adatok (valamint a tesztadatok) közötti különbségekről, olvassa el Jason Brownlee [ezt a cikket](https://machinelearningmastery.com/difference-test-validation-datasets/).

## Az adatok betöltése a memóriába (Keras segítségével)

Számos [mélytanulási keretrendszer](https://developer.nvidia.com/deep-learning-frameworks) létezik, mindegyiknek megvannak a maga érdemei. Ebben a workshopban a [Tensorflow 2](https://www.tensorflow.org/tutorials/quickstart/beginner), és konkrétan a [Keras API](https://keras.io/) segítségével fogunk dolgozni. A Keras számos hasznos beépített függvényt tartalmaz, amelyeket a számítógépes látás feladataihoz terveztek. Az [olvashatósága](https://blog.pragmaticengineer.com/readable-code/) és hatékonysága miatt professzionális környezetben is legitim választás a mélytanuláshoz, bár nem egyedül áll ebben a tekintetben, és érdemes többféle keretrendszert is megvizsgálni, amikor egy mélytanulási projektbe kezdünk.

A Keras által biztosított számos hasznos funkció egyike a számos segédmódszert tartalmazó modulok [számos gyakori adatkészlethez](https://www.tensorflow.org/api_docs/python/tf/keras/datasets), köztük az MNIST-hez.

Kezdjük a Keras adatkészlet moduljának betöltésével az MNIST-hez:

In [ ]:
from tensorflow.keras.datasets import mnist

Az `mnist` modullal egyszerűen betölthetjük az MNIST adatokat, amelyek már képekre és címkékre vannak felosztva mind a képzéshez, mind a validáláshoz:

In [ ]:
# az adatok, felosztva a training és a validáló készletekre
(x_train, y_train), (x_valid, y_valid) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


## Az MNIST adatok feltárása

Fentebb megállapítottuk, hogy az MNIST-adatkészlet 70 000 szürkeárnyalatos képet tartalmazott kézzel írt számjegyekről. A következő cellákat végrehajtva láthatjuk, hogy a Keras ezekből a képekből 60 000-et partícionált a képzéshez, és 10 000-et a validáláshoz (a képzés után), valamint azt is, hogy minden egyes kép maga egy 2D-s tömb, 28x28-as méretekkel:

In [ ]:
x_train.shape

(60000, 28, 28)

In [ ]:
x_valid.shape

(10000, 28, 28)

Továbbá láthatjuk, hogy ezek a 28x28-as képek 0 és 255 közötti előjel nélküli 8 bites egész számértékek gyűjteménye, amelyek megfelelnek a pixel szürkeárnyalatos értékének, ahol a "0" fekete, a "255" fehér, az összes többi érték pedig a kettő között van:

In [ ]:
x_train.dtype

dtype('uint8')

In [ ]:
x_train.min()

0

In [ ]:
x_train.max()

255

In [ ]:
x_train[0]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
         18,  18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,
          0,   0],
       [  

A [Matplotlib](https://matplotlib.org/) segítségével megjeleníthetjük az egyik ilyen szürkeárnyalatos képet az adathalmazunkban:

In [ ]:
import matplotlib.pyplot as plt

image = x_train[0]
plt.imshow(image, cmap='gray')

Így most már láthatjuk, hogy ez egy 28x28 pixeles kép egy 5-ösről. Vagy inkább egy 3-asról van szó? A választ az `y_train` adatban találjuk, amely az adatok helyes címkéit tartalmazza. Vessünk rá egy pillantást:

In [ ]:
y_train[0]

## Az adatok előkészítése a tanításhoz

A mélytanulásban gyakori, hogy az adatokat át kell alakítani, hogy a képzéshez ideális állapotba kerüljenek. Ennél a konkrét képosztályozási problémánál 3 feladatot kell elvégeznünk az adatokkal a képzés előkészítése során:
1. Laposítsuk a képadatokat, hogy egyszerűsítsük a kép bemenetét a modellbe.
2. Normalizáljuk a képadatokat, hogy a modell számára könnyebbé tegyük a kép bemeneti értékeinek feldolgozását.
3. A címkék kategorizálása, hogy a modell számára könnyebb legyen a címkeértékekkel dolgozni.

### A képadatok kiegyenlítése

Bár lehetséges, hogy egy mély tanulási modell elfogad egy 2 dimenziós képet (esetünkben 28x28 pixel), mi leegyszerűsítjük a dolgokat, és [átformáljuk](https://www.tensorflow.org/api_docs/python/tf/reshape) minden egyes képet egyetlen 784 folyamatos pixelből álló tömbre (megjegyzés: 28x28 = 784). Ezt nevezik a kép ellapításának is.

Itt ezt a `reshape` segédmetódus segítségével valósítjuk meg:

In [ ]:
x_train = x_train.reshape(60000, 784)
x_valid = x_valid.reshape(10000, 784)

Megerősíthetjük, hogy a képadatok átformálódtak, és most 1D-s tömbök gyűjteménye, amelyek egyenként 784 pixelértéket tartalmaznak:

In [ ]:
x_train.shape

In [ ]:
x_train[0]

### A képadatok normalizálása

A mélytanulási modellek jobban tudnak bánni a 0 és 1 közötti lebegőpontos számokkal (erről a témáról később). Az egész számok 0 és 1 közötti lebegőpontos értékekké való átalakítását [normalizálásnak] (https://developers.google.com/machine-learning/glossary#normalization) nevezik, és egy egyszerű megközelítés, amit itt az adatok normalizálására fogunk alkalmazni, az lesz, hogy az összes pixelértéket (amelyek, ha emlékszik, 0 és 255 között vannak) 255-tel osztjuk:

In [ ]:
x_train = x_train / 255
x_valid = x_valid / 255 

Most láthatjuk, hogy az értékek mind lebegőpontos értékek a "0.0" és "1.0" értékek között:

In [ ]:
x_train.dtype

In [ ]:
x_train.min()

In [ ]:
x_train.max()

### Kategorikus kódolás

Gondoljunk bele egy pillanatra, ha azt kérdeznénk, hogy mi a 7 - 2? Ha azt mondjuk, hogy a válasz 4, az közelebb áll ahhoz, mintha azt mondanánk, hogy a válasz 9. Azonban ennél a képosztályozási problémánál nem akarjuk, hogy a neurális hálózat megtanulja ezt a fajta érvelést: csak azt akarjuk, hogy válassza ki a helyes kategóriát, és értse meg, hogy ha van egy képünk az 5-ös számról, akkor a 4-es kitalálása ugyanolyan rossz, mint a 9-es kitalálása.

A jelenlegi helyzetben a képek címkéi 0 és 9 közötti egész számok. Mivel ezek az értékek egy számtartományt képviselnek, a modell megpróbálhat következtetéseket levonni a teljesítményéről az alapján, hogy milyen közel van a helyes számkategóriához, amit kitalál.

Ezért az adatainkkal valami olyasmit fogunk csinálni, amit kategorikus kódolásnak nevezünk. Ez a fajta átalakítás úgy módosítja az adatokat, hogy minden egyes érték az összes lehetséges kategória gyűjteménye legyen, azzal a tényleges kategóriával, amelyet az adott érték igaznak állít be.

Egyszerű példaként gondoljunk arra, hogy 3 kategóriánk van: piros, kék és zöld. Egy adott szín esetében e kategóriák közül 2 hamis lenne, a másik pedig igaz:

|Actual Color| Is Red? | Is Blue? | Is Green?|
|------------|---------|----------|----------|
|Red|True|False|False|
|Green|False|False|True|
|Blue|False|True|False|
|Green|False|False|True|

Ahelyett, hogy az "Igaz" vagy "Hamis" kifejezést használnánk, ugyanezt binárisan is ábrázolhatnánk, 0 vagy 1 értékkel:

|Actual Color| Is Red? | Is Blue? | Is Green?|
|------------|---------|----------|----------|
|Red|1|0|0|
|Green|0|0|1|
|Blue|0|1|0|
|Green|0|0|1|

Ez a kategorikus kódolás lényege, hogy a kategorikus címkékként értelmezendő értékeket olyan reprezentációvá alakítjuk át, amely a modell számára egyértelművé teszi kategorikus jellegüket. Ha tehát ezeket az értékeket használnánk a képzéshez, akkor átalakítanánk...

```python
values = ['red, green, blue, green']
```

... amit egy neurális hálózat nagyon nehezen tudna értelmezni, ehelyett:

```python
values = [
    [1, 0, 0],
    [0, 0, 1],
    [0, 1, 0],
    [0, 0, 1]
]
```

### A címkék kategorikus kódolása

A Keras biztosít egy segédprogramot a [kategorikus értékek kódolása](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical) számára, és itt ezt használjuk a kategorikus kódolás elvégzésére mind a képzési, mind a validálási címkék esetében:

In [ ]:
import tensorflow.keras as keras
num_categories = 10

y_train = keras.utils.to_categorical(y_train, num_categories)
y_valid = keras.utils.to_categorical(y_valid, num_categories)

Itt van a képzési címkék első 10 értéke, amelyeket most már kategorikusan kódoltunk:

In [ ]:
y_train[0:9]

## Creating the Model

With the data prepared for training, it is now time to create the model that we will train with the data. This first basic model will be made up of several *layers* and will be comprised of 3 main parts:

1. An input layer, which will receive data in some expected format
2. Several [hidden layers](https://developers.google.com/machine-learning/glossary#hidden-layer), each comprised of many *neurons*. Each [neuron](https://developers.google.com/machine-learning/glossary#neuron) will have the ability to affect the network's guess with its *weights*, which are values that will be updated over many iterations as the network gets feedback on its performance and learns
3. An output layer, which will depict the network's guess for a given image

### Instantiating the Model

To begin, we will use Keras's [Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) model class to instantiate an instance of a model that will have a series of layers that data will pass through in sequence:

In [ ]:
from tensorflow.keras.models import Sequential

model = Sequential()

### Creating the Input Layer

Next, we will add the input layer. This layer will be *densely connected*, meaning that each neuron in it, and its weights, will affect every neuron in the next layer. To do this with Keras, we use Keras's [Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense) layer class.

In [ ]:
from tensorflow.keras.layers import Dense

The `units` argument specifies the number of neurons in the layer. We are going to use `512` which we have chosen from experimentation. Choosing the correct number of neurons is what puts the "science" in "data science" as it is a matter of capturing the statistical complexity of the dataset. Try playing around with this value later to see how it affects training and to start developing a sense for what this number means.

We will learn more about activation functions later, but for now, we will use the `relu` activation function, which in short, will help our network to learn how to make more sophisticated guesses about data than if it were required to make guesses based on some strictly linear function.

The `input_shape` value specifies the shape of the incoming data which in our situation is a 1D array of 784 values:

In [ ]:
model.add(Dense(units=512, activation='relu', input_shape=(784,)))

### Creating the Hidden Layer

Now we will add an additional densely connected layer. Again, much more will be said about these later, but for now know that these layers give the network more parameters to contribute towards its guesses, and therefore, more subtle opportunities for accurate learning:

In [ ]:
model.add(Dense(units = 512, activation='relu'))

### Creating the Output Layer

Finally, we will add an output layer. This layer uses the activation function `softmax` which will result in each of the layer's values being a probability between 0 and 1 and will result in all the outputs of the layer adding to 1. In this case, since the network is to make a guess about a single image belonging to 1 of 10 possible categories, there will be 10 outputs. Each output gives the model's guess (a probability) that the image belongs to that specific class:

In [ ]:
model.add(Dense(units = 10, activation='softmax'))

### Summarizing the Model

Keras provides the model instance method [summary](https://www.tensorflow.org/api_docs/python/tf/summary) which will print a readable summary of a model:

In [ ]:
model.summary()

Note the number of trainable parameters. Each of these can be adjusted during training and will contribute towards the trained model's guesses.

### Compiling the Model

Again, more details are to follow, but the final step we need to do before we can actually train our model with data is to [compile](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential#compile) it. Here we specify a [loss function](https://developers.google.com/machine-learning/glossary#loss) which will be used for the model to understand how well it is performing during training. We also specify that we would like to track `accuracy` while the model trains:

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

## Training the Model

Now that we have prepared training and validation data, and a model, it's time to train our model with our training data, and verify it with its validation data.

"Training a model with data" is often also called "fitting a model to data." Put this latter way, it highlights that the shape of the model changes over time to more accurately understand the data that it is being given.

When fitting (training) a model with Keras, we use the model's [fit](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit) method. It expects the following arguments:

* The training data
* The labels for the training data
* The number of times it should train on the entire training dataset (called an *epoch*)
* The validation or test data, and its labels

Run the cell below to train the model. We will discuss its output after the training completes:

In [ ]:
history = model.fit(
    x_train, y_train, epochs=5, verbose=1, validation_data=(x_valid, y_valid)
)

### Observing Accuracy

For each of the 5 epochs, notice the `accuracy` and `val_accuracy` scores. `accuracy` states how well the model did for the epoch on all the training data. `val_accuracy` states how well the model did on the validation data, which if you recall, was not used at all for training the model.

The model did quite well! The accuracy quickly reached close to 100%, as did the validation accuracy. We now have a model that can be used to accurately detect and classify hand-written images.

The next step would be to use this model to classify new not-yet-seen handwritten images. This is called [inference](https://blogs.nvidia.com/blog/2016/08/22/difference-deep-learning-training-inference-ai/). We'll explore the process of inference in a later exercise. 

## Summary

It's worth taking a moment to appreciate what we've done here. Historically, the expert systems that were built to do this kind of task were extremely complicated, and people spent their careers building them (check out the references on the [official MNIST page](http://yann.lecun.com/exdb/mnist/) and the years milestones were reached).

MNIST is not only useful for its historical influence on Computer Vision, but it's also a great [benchmark](http://www.cs.toronto.edu/~serailhydra/publications/tbd-iiswc18.pdf) and debugging tool. Having trouble getting a fancy new machine learning architecture working? Check it against MNIST. If it can't learn on this dataset, chances are it won't learn on more complicated images and datasets.

## Clear the Memory

Before moving on, please execute the following cell to clear up the GPU memory. This is required to move on to the next notebook.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Next

In this section you learned how to build and train a simple neural network for image classification. In the next section, you will be asked to build your own neural network and perform data preparation to solve a different image classification problem.

## ☆ Bonus Exercise ☆

Have time to spare? In the next section, we will talk about how we arrived at some of the numbers above, but we can try imagining what it was like to be a researcher developing the techniques commonly used today.

Ultimately, each neuron is trying to fit a line to some data. Below, we have some datapoints and a randomly drawn line using the equation [y = mx + b](https://www.mathsisfun.com/equation_of_line.html).

Try changing the `m` and the `b` in order to find the lowest possible loss. How did you find the best line? Can you make a program to follow your strategy?

In [ ]:
import numpy as np
from numpy.polynomial.polynomial import polyfit
import matplotlib.pyplot as plt

m = -2  # -2 to start, change me please
b = 40  # 40 to start, change me please

# Sample data
x = np.array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9])
y = np.array([10, 20, 25, 30, 40, 45, 40, 50, 60, 55])
y_hat = x * m + b

plt.plot(x, y, '.')
plt.plot(x, y_hat, '-')
plt.show()

print("Loss:", np.sum((y - y_hat)**2)/len(x))

Have an idea? Excellent! Please shut down the kernel before moving on.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)